### **Load Environment variables from .env file**

In [3]:
from langchain.chat_models import AzureChatOpenAI
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown

load_dotenv()
# env variables that are used by LangChain
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_TYPE'] = "azure"
os.environ['OPENAI_API_VERSION'] = os.getenv("OPENAI_DEPLOYMENT_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_DEPLOYMENT_VERSION")
openai.api_base = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
openai.api_key = os.getenv("OPENAI_API_KEY")

### **Initialize the LLM model which is deployed in Azure with LangChain**

In [4]:

def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             temperature=0,
             max_tokens=400,
             stop="<|im_end|>",
             ):
    # AzureOpenAI
    llm = AzureChatOpenAI(deployment_name=deployment_name,
                          model=model,
                          temperature=temperature,
                          max_tokens=max_tokens,
                          model_kwargs={"stop": ["<|im_end|>"]})
    return llm

### **Add personality to the model and start asking questions**
We call directly the Azure OpenAI API with ***ChatCompletion*** API

In [5]:
# prepare prompt
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer in a clear and coincise manner."},
            {"role": "user", "content": "Good morning, how are you today?"}]


answer = openai.ChatCompletion.create(engine=OPENAI_DEPLOYMENT_NAME,

                                      messages=messages,)
display(HTML("ChatCompletion (gpt-35-turbo) :" +
        answer.choices[0].message.content))

In [6]:
# prepare prompt with another question:
messages = [{"role": "system", "content": "You are q HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
            {"role": "user", "content": "What's string theory?"}]


answer = openai.ChatCompletion.create(engine=OPENAI_DEPLOYMENT_NAME,
                                      messages=messages,)

display(HTML(answer.choices[0].message.content))

In [7]:
# prepare prompt with another question:
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer as for a FIVE YEARS old child."},
            {"role": "user", "content": "what's string theory?"}]


answer = openai.ChatCompletion.create(engine=OPENAI_DEPLOYMENT_NAME,
                                      messages=messages,)

# print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)
display(HTML(answer.choices[0].message.content))

### **LangChain**

LangChain is a framework built around Large Language Models (LLMs).

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs.

LangChain components: Models, Prompts, Indexes(Document Loaders and Splitters, Vector DBs), Chains and Agents

In [8]:
# init LLM Azure OpenAI model
llm = init_llm()
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.0, model_kwargs={'stop': ['<|im_end|>']}, openai_api_key='aa87dc8868f34c15b8c299c036c347b6', openai_api_base='https://openailabs303474.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=400, tiktoken_model_name=None, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_version='2023-05-15')

In [9]:
# Import LangChain schema classes for messages
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [10]:
messages = [
    SystemMessage(
        content="You're nice and happy assistant. Answer happily users questions.")
]
messages.append(
    HumanMessage(content="Good morning, how are you today?"))

answer = llm(messages)

display(HTML(f"gpt-35-turbo: {answer.content}"))

In [11]:

messages = [
    SystemMessage(
        content="You're nice and happy assistant for Python developer assistant. Assist the developers with Python requests. ")
]
messages.append(
    HumanMessage(content="Create a Python function that takes a string argument and reverses it. Generate just one version of the function."))

answer = llm(messages)
display(HTML(f"gpt-35-turbo: {answer.content}"))

#### Prompts with LangChain

##### ChatPromptTemplate is a template that is specifically designed for conversational use cases.

It infers the variables from the template, so no need to pass them in as arguments.

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# init model
llm = init_llm()

# create template for prompt
template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            ('''You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: It's not clear or the question is not related to {expertise}. ''')
        ),
        HumanMessagePromptTemplate.from_template('''{question}'''),
    ]
)

answer = llm(template.format_messages(profession="Financial Trading Consultant",  expertise="Risk Management",
                                      question="How do you assess the risk tolerance of a new client?"))
display(Markdown("gpt-35-turbo: " + answer.content))

gpt-35-turbo: To assess the risk tolerance of a new client, you can use a risk assessment questionnaire. This questionnaire will ask the client a series of questions about their investment goals, time horizon, financial situation, and risk preferences. Based on the client's responses, you can determine their risk tolerance and recommend an appropriate investment strategy. It's important to note that risk tolerance can change over time, so it's important to reassess periodically to ensure the investment strategy remains appropriate.

In [13]:
# asking not related question and reusing the same template
answer = llm(template.format_messages(profession="Financial Trading Consultant",  expertise="Risk Management",
                                      question="What's the fastest car in the world? Answer in one sentence."))

display(Markdown(answer.content))

It's not related to Risk Management.

#### **OPTIONAL.** LLM output parsers with LangChain
You can define the output schema and LangChain will parse the output for you.

The main idea is to define parsing instructions as a code rather than doing it in textual form.


In [14]:

customer_call = """

**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you..."""

In [16]:

# init model
llm = init_llm()

prompt = [
    SystemMessage(content=f"""For the following text, extract the following information:
                agent politeness: How polite is the agent? use the following values to descibe agent politenes: very polite, polite, neutral, impolite, very impolite.
                agent knowledge: How knowledgeable is the agent? use the following values to descibe agent knowledge: very knowledgeable, knowledgeable, neutral, not knowledgeable, very not knowledgeable.
                customer issue resolution: How well did the agent resolve the issue? use the following values to descibe issue resolution: very well, well, neutral, not well, very not well.
                customer satisfaction: How satisfied is the customer? use the following values to descibe customer satisfaction: very satisfied, satisfied, neutral, dissatisfied, very dissatisfied.

                Format the output as a json object with the following keys: agent_politeness, agent_knowledge, customer_issue_resolution, customer_satisfaction.

                text: {customer_call}""")
]

# Note that the type of the response is a string, it looks like a json object, but it's string. We're looking for more typed response.
response = llm(prompt)

display(Markdown(response.content))

{
    "agent_politeness": "polite",
    "agent_knowledge": "knowledgeable",
    "customer_issue_resolution": "well",
    "customer_satisfaction": "satisfied"
}

##### Define the output schema

In [17]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [18]:

agent_politeness_schema = ResponseSchema(
    name="politeness", description="How polite is the agent?")
aggent_knowledge_schema = ResponseSchema(
    name="knowledge", description="How knowledgeable is the agent?")
customer_issue_resolution_schema = ResponseSchema(
    name="customer_issue_resolution", description="How well did the agent resolve the issue?")
customer_satisfaction_schema = ResponseSchema(
    name="customer_satisfaction", description="How satisfied is the customer?")

customer_service_schemas = [agent_politeness_schema, aggent_knowledge_schema,
                            customer_issue_resolution_schema, customer_satisfaction_schema]

output_parser = StructuredOutputParser.from_response_schemas(
    customer_service_schemas)

# get a string that contains instructions for how the response should be formatted, and then insert that into your prompt.
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"politeness": string  // How polite is the agent?
	"knowledge": string  // How knowledgeable is the agent?
	"customer_issue_resolution": string  // How well did the agent resolve the issue?
	"customer_satisfaction": string  // How satisfied is the customer?
}
```


In [19]:
# adding format instructions to the prompt:
formatted_prompt = [
    SystemMessage(content=f"""For the following text, extract the following information:
                agent politeness: How polite is the agent? use the following values to descibe agent politenes: very polite, polite, neutral, impolite, very impolite.
                agent knowledge: How knowledgeable is the agent? use the following values to descibe agent knowledge: very knowledgeable, knowledgeable, neutral, not knowledgeable, very not knowledgeable.
                customer issue resolution: How well did the agent resolve the issue? use the following values to descibe issue resolution: very well, well, neutral, not well, very not well.
                customer satisfaction: How satisfied is the customer? use the following values to descibe customer satisfaction: very satisfied, satisfied, neutral, dissatisfied, very dissatisfied.

                Format the output as a json object with the following keys: agent_politeness, agent_knowledge, customer_issue_resolution, customer_satisfaction.

                Text: {customer_call}
                Output format instructions: {format_instructions}
                """)
]

In [20]:
# run model
response = llm(formatted_prompt)

In [22]:
print(type(response))
display(Markdown(response.content))

<class 'langchain.schema.messages.AIMessage'>


```json
{
	"politeness": "very polite",
	"knowledge": "knowledgeable",
	"customer_issue_resolution": "well",
	"customer_satisfaction": "satisfied"
}
```

In [23]:
# output of parser is a dictionary, which is very easy to work with after you get the output from the model
dict = output_parser.parse(response.content)
dict.get("politeness")

'very polite'

#### Prompts management best practices
Reuse prompts as much as possible. This will help you to get more consistent results.

Treat your prompts as a code, keep it in a version control system. This will help you to track changes and to revert them if needed.

Use LangChain specific to use case prompt templates, e.g. ChatPromptTemplate for conversational flows.


###  **One-shot, Few-shot learning**

This technique could improve model performance by a lot. 
We can use the model to learn from a few examples and then use it to generate text. This is called few-shot learning. We can also use the model to learn from a single example and then use it to generate text. This is called one-shot learning.

In [24]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# init model
llm = init_llm()
# create template for prompt

template_few_shot = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            ("""You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that a user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
           
            HUMAN: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.
            
            HUMAN: Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I'd recommended that a client diversify their portfolio further to reduce risk exposure.
            
            HUMAN: How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
            ASSISTANT: I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance. 
            """)
        ),
        HumanMessagePromptTemplate.from_template('''{question}'''),
    ]
)

chain = LLMChain(llm=llm, prompt=template_few_shot)

answer = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management",
                   question="How do you use technology or specific financial tools to assist in risk management for your clients?")
display(Markdown(answer))

There are various financial tools and technologies that can be used to assist in risk management for clients. For example, risk management software can be used to analyze and monitor market trends, identify potential risks, and provide real-time alerts to clients. Additionally, financial modeling tools can be used to simulate different market scenarios and assess the potential impact on a client's portfolio. Other tools such as stop-loss orders and limit orders can be used to automatically trigger trades when certain price levels are reached, helping to minimize losses and manage risk. As a consultant, I would assess the specific needs of each client and recommend the appropriate tools and technologies to help manage their risk effectively.

The Large Language Models (LLMs) are stateless. This means that they don’t retain any information about the conversation history.

In [25]:
# Since we don't save a history of the conversation, the last is not listed...
res = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management",
                question="List all questions that I asked you about risk management.")
display(Markdown(res))

1. How do you assess the risk tolerance of a new client?
2. Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
3. How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?

### **Retain conversation history** 

##### The Large Language Models (LLMs) are stateless. This means that they don’t retain any information about the conversation history.
Each transaction is independent of the previous one. Chatbots keep in memory the conversation history and use it to generate the next response. This is why they are able to generate more coherent responses.

Previously we saw that a model fails to answer the question that requires context. We can solve this problem by retaining the conversation history. We can do this by using the LangChain ConversationBufferMemory.

In [26]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

llm = init_llm()

# ConversationBufferMemory is a memory that stores the conversation history
memory = ConversationBufferMemory()
# try to change the verbose to True, to see more details
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

In [27]:
answer = conversation.run(input=template_few_shot.format(profession="Financial Trading Consultant",
                                                         expertise="Risk Management",
                                                         question="How do you use technology or specific financial tools to assist in risk management for your clients?"))
display(Markdown(answer))

There are a variety of tools and technologies that I use to assist in risk management for my clients. For example, I might use risk assessment software to help identify potential risks and develop strategies to mitigate them. I might also use financial modeling tools to help clients understand the potential impact of different investment scenarios on their portfolio. Additionally, I might use data analytics tools to help identify trends and patterns in market data that can inform investment decisions. Overall, technology plays a critical role in helping me provide effective risk management advice to my clients.

In [28]:
# Now with the conversation history, the model can answer questions that require context.
answer = conversation.run(input=template_few_shot.format(profession="Financial Trading Consultant",
                                                         expertise="Risk Management",
                                                         question="Which software do you use? List software products in a separate line."))

display(Markdown(answer))

There are several software products that I use for risk management, including:

- Riskalyze
- Morningstar Direct
- BlackRock Aladdin
- Bloomberg Terminal
- FactSet
- Thomson Reuters Eikon

Each of these tools has its own strengths and can be used to help identify and manage different types of risks.

In [32]:
# Now with the conversation history, the model can answer questions that require context.
answer = conversation.run(input=template_few_shot.format(profession="Financial Trading Consultant",
                                                         expertise="Risk Management",
                                                         question="Construct a comparison table detailing the advantages and disadvantages of the aforementioned software tools, including cost."))
display(Markdown(answer))

Sure, here is a comparison table outlining the advantages, disadvantages, and cost of the software tools I mentioned earlier:

| Software Tool | Advantages | Disadvantages | Cost |
| --- | --- | --- | --- |
| Riskalyze | Easy to use, customizable risk assessment | Limited investment analysis capabilities | $99-$199/month |
| Morningstar Direct | Comprehensive investment analysis, robust data | Expensive, steep learning curve | $24,000-$36,000/year |
| BlackRock Aladdin | Powerful risk management tools, integrates with other systems | Expensive, complex implementation | Custom pricing |
| Bloomberg Terminal | Wide range of financial data and news, customizable | Expensive, requires training | $20,000/year |
| FactSet | Comprehensive data and analysis, customizable | Expensive, requires training | $20,000-$50,000/year |
| Thomson Reuters Eikon | Wide range of financial data and news, customizable | Expensive, requires training | Custom pricing |

Overall, each tool has its own strengths and weaknesses, and the cost will depend on the specific needs and preferences of the client. It's important to carefully evaluate each tool before making a decision, taking into account both the features and the cost.

In [50]:
# you can print the conversation history buffer:
history = conversation.memory.chat_memory.messages
for msg in history:
    print(f"{msg.type}: {msg.content}")

human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that a user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
           
            HUMAN: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.
            
            HUMAN: Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I'd recommended that a client diversify their portfolio further to reduce risk exposure.
            
            HUMAN: How do

In [51]:
# pay attention that LangChain added to the prompt: ```Current conversation:{history}```
print(conversation.prompt)
print()
print(memory.load_memory_variables({}))

input_variables=['history', 'input'] output_parser=None partial_variables={} template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:' template_format='f-string' validate_template=True

{'history': "Human: You are a Financial Trading Consultant answering users questions. \n            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that a user is not a subject expert.\n            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.\n           \n            HUMAN: How do you assess the risk tolerance of a new client?\n            ASSISTANT: I begin by having a comprehensive discussion with the client about their finan

In [33]:
answer = conversation.run(input=template_few_shot.format(profession="Financial Trading Consultant",
                                                         expertise="Risk Management",
                                                         question="List all questions I've asked you about Risk Management?"))
display(Markdown(answer))

Sure, here are all the questions you've asked me about Risk Management:

1. How do you assess the risk tolerance of a new client?
2. Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
3. How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
4. How do you use technology or specific financial tools to assist in risk management for your clients?
5. Which software do you use? List software products in a separate line.
6. Create a table of Pros/Cons of the above software tool.
7. Construct a comparison table detailing the advantages and disadvantages of the aforementioned software tools.
8. Construct a comparison table detailing the advantages and disadvantages of the aforementioned software tools, including cost.

#### More conversation memory types.

Keeping full conversation history could be expensive and we can hit the Azure API limits. LangChain provides different types of conversation memory that can be used to keep the conversation history and mitigate the limits issues.

**1. ConversationBufferWindowMemory** - keeps the last N messages in the conversation history.

**2. ConversationTokenBufferMemory** - keeps the last N tokens in the conversation history.

**3. ConversationSummaryBufferMemory** - keeps summary of the conversation over time.

Additional Memory Type includes:

**4. Vector data memory** - stires the text in a vector DB and retrieves most semantically similar text.

**5. Entity memories** - use LLM which rememebers details about specific entities. 

Note: You can use multiple memories at the same time. 

Finally you can store the conevrsation history in a conventional database like SQL or NoSQL. 


In [22]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "What's your name"},
                    {"output": "My name is John"})

memory.load_memory_variables({})

{'history': "Human: What's your name\nAI: My name is John"}